In [12]:
import tensorflow as tf
shakespeare_url = "https://homl.info/shakespeare" # shortcut URL
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
     shakespeare_text = f.read()

In [13]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

# Print each token along with its index
for token, index in tokenizer.word_index.items():
    print(f"Token: {token}, Index: {index}")

Token:  , Index: 1
Token: e, Index: 2
Token: t, Index: 3
Token: o, Index: 4
Token: a, Index: 5
Token: i, Index: 6
Token: h, Index: 7
Token: s, Index: 8
Token: r, Index: 9
Token: n, Index: 10
Token: 
, Index: 11
Token: l, Index: 12
Token: d, Index: 13
Token: u, Index: 14
Token: m, Index: 15
Token: y, Index: 16
Token: w, Index: 17
Token: ,, Index: 18
Token: c, Index: 19
Token: f, Index: 20
Token: g, Index: 21
Token: b, Index: 22
Token: p, Index: 23
Token: :, Index: 24
Token: k, Index: 25
Token: v, Index: 26
Token: ., Index: 27
Token: ', Index: 28
Token: ;, Index: 29
Token: ?, Index: 30
Token: !, Index: 31
Token: -, Index: 32
Token: j, Index: 33
Token: q, Index: 34
Token: x, Index: 35
Token: z, Index: 36
Token: 3, Index: 37
Token: &, Index: 38
Token: $, Index: 39


In [14]:
max_id = len(tokenizer.word_index)
dataset_size = len(tokenizer.texts_to_sequences([shakespeare_text])[0])
print(dataset_size)

1115394


In [15]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [16]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
print(train_size)

1003854


In [17]:
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [18]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [19]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [20]:
dataset = dataset.map(
 lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [21]:
model = tf.keras.models.Sequential([
 tf.keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
 dropout=0.2, recurrent_dropout=0.2),
 tf.keras.layers.GRU(128, return_sequences=True,
 dropout=0.2, recurrent_dropout=0.2),
 tf.keras.layers.Dense(max_id,
 activation="softmax")
 ])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=20)